In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope

# Define the custom F1 score function
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

model_path = '/Users/sakshamjain/Desktop/Projects/FER/projects and models/CNNMODELfinal.keras'

try:
    with custom_object_scope({'f1_score': f1_score}):
        model = load_model(model_path, compile=False)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

# Print model summary to inspect layers and weights
model.summary()

#use python 3.11.5
import tensorflow as tf
print(tf.__version__)


Model loaded successfully.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 42, 42, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 40, 40, 32)        36896     
                                                                 
 dense (Dense)               (None, 40, 40, 32)        1056      
                                                                 
 dropout (Dropout)           (None, 40, 40, 32)        0         
                                                                 
 batch_normalization (Batch  

In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


def preprocess_face(img):
    img = cv2.resize(img, (48, 48))
    img = img / 255.0  
    return np.expand_dims(img, axis=0)


cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    if not ret:
        break


    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))


    for (x, y, w, h) in faces:

        face_roi = frame[y:y+h, x:x+w]


        processed_face = preprocess_face(face_roi)


        prediction = model.predict(processed_face)
        expression_idx = np.argmax(prediction)


        expression_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
        expression = expression_labels[expression_idx]


        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)


        cv2.putText(frame, expression, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)


    cv2.imshow('Expression Prediction', frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 14ms/step


2024-12-19 00:49:16.205 Python[81829:7925877] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-19 00:49:16.205 Python[81829:7925877] +[IMKInputSession subclass]: chose IMKInputSession_Modern


1/1 [==============================] - 0s 12ms/step
